# Imports

In [17]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from google.cloud import bigquery
import os
from datetime import date
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

True

In [18]:
"""
    parameters received from papermill

    these are the parameters that are passed to the script
    above is the default values
    they can be overridden by the user when running the script
"""

BASE_URL = "https://www.gov.br/anp/pt-br"
B100_SALES = f"{BASE_URL}/assuntos/distribuicao-e-revenda/comercializacao-de-biodiesel"
RAW_PATH = "data/raw/b100_sales"

bucket_name = os.getenv("GOOGLE_BUCKET_NAME")
start_date = "2023-01-01"
end_date = "2023-12-31"

file_path = f"gs://{bucket_name}/data/raw/b100_sales/b100_sales_{start_date}_{end_date}.xlsx"

In [19]:
"""
Get file download URL
"""

response = requests.get(B100_SALES, verify=False)
soup = BeautifulSoup(response.content, "html.parser")
if soup is None:
    raise Exception("Failed to retrieve data from the URL.")
title_text = 'Volumes Comercializados de Biodiesel'
title_h3 = soup.find('h3', string=title_text)
jump_p = title_h3.find_next_sibling()
jump_p = jump_p.find_next_sibling()
year = jump_p.find_next_sibling()

data_by_year = {}

current_year = year.get_text(strip=True)
data_by_year[current_year] = []

next_elem = year.find_next_sibling()
while next_elem:
    if next_elem.name == 'ul':
        for li in next_elem.find_all('li'):
            a_tag = li.find('a')
            if a_tag and 'href' in a_tag.attrs:
                link = a_tag['href']
                text = a_tag.get_text(strip=True)
                li_text = li.get_text(strip=True)


                start = li_text.find('Atualizado em ')
                if start != -1:
                    li_text = li_text[start + len('Atualizado em '):-1].strip()

                data_by_year[current_year].append({'text': text, 'link': link, 'updated_date': li_text})
            else:
                print("Elemento <li> nÃ£o contÃ©m um link.")
        next_elem = next_elem.find_next_sibling()
    else:
        if next_elem.name == 'h3' and next_elem.get_text(strip=True).isdigit():
            current_year = next_elem.get_text(strip=True)
            if current_year not in data_by_year:
                data_by_year[current_year] = []
            next_elem = next_elem.find_next_sibling()
            continue
        else:
            break


/Users/dwbessa/projects/vibra/pipeline_biomass_calculation/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [20]:
current_year = start_date.split("-")[0] if start_date else date.today().year
download_url =  data_by_year[current_year][0]['link']

response = requests.get(download_url, verify=False)
file_name = download_url.split("/")[-1]
if response.status_code != 200:
    raise Exception(f"Falha ao baixar o arquivo: {response.status_code}")

file_bytes = response.content
file_buffer = BytesIO(file_bytes)

/Users/dwbessa/projects/vibra/pipeline_biomass_calculation/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [21]:
"""
    - read the data from the file
    - convert it to a pandas dataframe
    - rename columns to lowercase and snake_case

    B100_BRONZE_COLUMNS_MAPPING is a dictionary that maps the columns in the Excel file to the columns in the DataFrame
"""
B100_BRONZE_COLUMNS_MAPPING = {
    "2023": {
        "Mês/Ano": "date",
        "Raiz\nCNPJ": "company_base_cnpj",
        "Razão Social": "company_name",
        "Quantidade\nde Produto\n(m³)": "volume_m3"
    },
    "2024": {
        "Data": "date",
        "Raiz de CNPJ do Distribuidor": "company_base_cnpj",
        "Razão Social do Distribuidor": "company_name",
        "Razão Social do Produtor": "producer_name",
        "CNPJ do Produtor": "producer_cnpj",
        "Volume (m3)": "volume_m3"
    }
}

map_to_use = B100_BRONZE_COLUMNS_MAPPING.get(current_year, B100_BRONZE_COLUMNS_MAPPING["2024"])
df = pd.read_excel(file_buffer, header=2, usecols=lambda x: 'Unnamed' not in x)
df = df.rename(columns=map_to_use)

In [22]:
"""
    change columns data types
"""

df['date'] = pd.to_datetime(df['date'], format='%Y%m')
df["company_base_cnpj"] = df["company_base_cnpj"].astype(str).str.zfill(8)
df['month'] = df['date'].dt.strftime('%Y-%m')
df['date'] = df['date'].dt.date  # convert back to date if needed
df['volume_m3'] = df['volume_m3'].astype(float)

if int(current_year) < 2024:
    df["producer_name"] = None
    df["producer_cnpj"] = None
else:
    df["producer_cnpj"] = df["producer_cnpj"].astype(str).str.zfill(14)

In [23]:
"""
    get all datas to insert into BigQuery
    remove empty dataframes
    iterate over the dataframe and append the data to the corresponding month datafram
    create a hashmap to store the dataframes for each month
"""
start_date_obj = pd.to_datetime(start_date).date()
end_date_obj = pd.to_datetime(end_date).date()

filter_by_start_and_end_date = (df['date'] >= start_date_obj) & (df['date'] <= end_date_obj)
df = df[filter_by_start_and_end_date]
df_by_month = {month: month_df.drop(columns='month') for month, month_df in df.groupby('month')}
print("Months available in the dictionary:", df_by_month.keys())



Months available in the dictionary: dict_keys(['2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12'])


In [24]:
"""
    insert data into BigQuery
"""

client = bigquery.Client()
project_id = os.getenv("GOOGLE_PROJECT_ID")
bq_dataset = "rw_ext_anp"
table_name = "rw_ext_anp_b100_sales"

table_id = f"{project_id}.{bq_dataset}.{table_name}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

for month, monthly_df in df_by_month.items():
    year = monthly_df['date'].iloc[0].year
    month_num = monthly_df['date'].iloc[0].month
    partition_key = f"{year}{month_num:02d}"

    print(f"Inserting data for partition: {partition_key}")
    job = client.load_table_from_dataframe(
        monthly_df, f"{table_id}${partition_key}", job_config=job_config
    )
    job.result()
    print(f"Data for {partition_key} inserted successfully.")


Inserting data for partition: 202301


BadRequest: 400 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/named-embassy-456813-f3/jobs?uploadType=multipart: Partitioning specification must be provided in order to create partitioned table